# LANL-Earthquake-Prediction

### Table of Contents
* [1. Introduction](#section1)
* [2. Initial Setup](#section2)
* [3. Training Set EDA](#section3)
* [4. Test Set EDA](#section4)
* [5. Models](#section5)
    * [5.1 Correlations](#section5.1)
    * [5.2 Spectrogram](#section5.2)
    * [5.3 Fourier Transform per Sampling Sequence of 4096 measurements](#section5.3)
    * [5.4 Wavelets Transform per Sampling Sequence of 4096 measurements](#section5.4)
        * [5.4.1. Continous Wavelet Transf](#section5.4.1)
        * [5.4.2. Discrete Wavelet Transform](#section5.4.2)
* [6. Evaluation](#section6)

<a id='section1'></a>
## 1. Introduction

TBD

<a id='section2'></a>
## 2. Initial Setup

In [ ]:
import numpy as np 
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style
style.use('ggplot')

import seaborn as sns
sns.set()

from IPython.display import HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

import timeit
from tqdm import tqdm

from ipywidgets import interact
import ipywidgets as widgets

from scipy import fftpack

from os import listdir
print(listdir("../input"))

<a id='section3'></a>
## 3. Training Set EDA

In [ ]:
train_nrows = !wc -l ../input/train.csv
train_nrows_val = int(train_nrows[0].split()[0])
print('train.csv contains {:,} rows'.format(train_nrows_val))

In [ ]:
!head ../input/train.csv

Question: what is the precision of the ttf in train.csv? See the value of `max_precision` below

In [ ]:
start_time = timeit.default_timer()
max_precision = 0
count = 0
with open('../input/train.csv', 'r') as f:
    while count<10: #True: #count <10:
        line = f.readline()
        if not line: 
            break
        else: 
            #print(line.rstrip())
            if count > 0:
                #print(line[:-1].split('.')[1])
                if '.' in line: 
                    str_len = len(line[:-1].split('.')[1])
                    #print(str_len)
                    if max_precision < str_len:
                        print(line)
                    max_precision = max_precision if max_precision > str_len else str_len
#            print(line)
#            print(line.split('.')[1])
        count +=1
print (max_precision)
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))    

In [ ]:
column_names = !head -n1 ../input/train.csv
print(column_names[0].split(','))

In [ ]:
df_train_sample = pd.read_csv('../input/train.csv', skiprows = 0, nrows=100,
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64}) #use chunksize to iterate

In [ ]:
def display_df_with_preset_precision(df, precision):
    curr_precision = pd.get_option("display.precision")
    pd.set_option("display.precision", precision)
    display(df)
    pd.set_option("display.precision", curr_precision)
    
display_df_with_preset_precision(df_train_sample.head(9), max_precision)

In [ ]:
!head ../input/train.csv

In [ ]:
#keep minimal mem footprint 
try:
    del(df_train_sample)    
except NameError:
    pass

Question: what are the time intervals between the samples?<br>
Question: what are the entries where the ttf jumpes up, but not down ? <br>
Question: what are the entries where the ttf jumpes down are long, but not down ?

In [ ]:
start_time = timeit.default_timer()
from collections import Counter
diff_ttf_2_counter_dict = Counter()
try:
    del(df_train_iter)    
except NameError:
    pass
#idx = 0
df_train_iter = pd.read_csv('../input/train.csv', chunksize=train_nrows_val//100,
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64},iterator=True)
time_to_failure_diffs_set = set()
df_after_jumping_up_points = pd.DataFrame()
df_after_long_jumps_down_points=pd.DataFrame()
for df in df_train_iter:
    df['diff_in_time_to_failure']=df['time_to_failure'].diff()
    nparr_ = df['diff_in_time_to_failure'].values
    nparr_ = nparr_[~np.isnan(nparr_)]
    diff_ttf_2_counter_dict += Counter(nparr_)

#    if idx == 0:
#        df_sample_0 = df.copy()
#    if idx == 1:
#        df_sample_1 = df.copy()        
#    idx = idx +1
    #time_to_failure_diffs_set = time_to_failure_diffs_set.union(list(set(df['diff_in_time_to_failure'].values)))
    #print(set(df['diff_in_time_to_failure'].values[:5]))
    df_jumps_up = df.loc[(df['diff_in_time_to_failure'] > 0)]
    df_long_jumps_down = df.loc[(df['diff_in_time_to_failure'] < -0.0001)]
    #display(df_jumps)
    df_after_jumping_up_points=df_after_jumping_up_points.append(df_jumps_up)
    df_after_long_jumps_down_points=df_after_long_jumps_down_points.append(df_long_jumps_down)
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))    

In [ ]:
len(list(diff_ttf_2_counter_dict.keys()))

In [ ]:
diff_ttf_2_counter_dict

In [ ]:
#positive jumbs
{k: v for k, v in diff_ttf_2_counter_dict.items() if k>0}

In [ ]:
10.**max_precision

In [ ]:
negative_diff_tff_2_counter_dict = Counter()

In [ ]:
for k, v in diff_ttf_2_counter_dict.items():
    if k<=0:
        negative_diff_tff_2_counter_dict[int(-k*(10.**max_precision))] += v 
negative_diff_tff_2_counter_dict     

In [ ]:
#time diff within sampling frames
time_diff_within_sampling_frames_2_count_dict = {k: v for k, v in negative_diff_tff_2_counter_dict.items() if k<100}
time_diff_within_sampling_frames_2_count_dict

In [ ]:
#time diff between sampling frames
time_diff_between_sampling_frames_2_count_dict = {round(k/(10.**max_precision),6): v for k, v in negative_diff_tff_2_counter_dict.items() if k>=100}
time_diff_between_sampling_frames_2_count_dict

In [ ]:
print(df_after_jumping_up_points.shape)
display_df_with_preset_precision(df_after_jumping_up_points, max_precision)

In [ ]:
display_df_with_preset_precision(df_after_long_jumps_down_points.head(),max_precision)

Question: what are the lengths of sampling frames (the numbers of samples between long jups down)?

In [ ]:
np.unique(np.diff(df_after_long_jumps_down_points.index))

In [ ]:
len(np.where(np.diff(df_after_long_jumps_down_points.index)==8192)[0])

In [ ]:
len(np.where(np.diff(df_after_long_jumps_down_points.index)==4095)[0])

In [ ]:
len(np.where(np.diff(df_after_long_jumps_down_points.index)==4096)[0])

In [ ]:
max(time_diff_within_sampling_frames_2_count_dict.keys())/(10.**max_precision)*8192

In [ ]:
np.mean(list(time_diff_between_sampling_frames_2_count_dict.keys()))

Question: what is the relation between the time length of the longest (8192 samples) samplling frame and the average time distance (0.001) between the sampling frames?

In [ ]:
(max(time_diff_within_sampling_frames_2_count_dict.keys())/(10.**max_precision)*8192)/np.mean(list(time_diff_between_sampling_frames_2_count_dict.keys()))

In [ ]:
start_time = timeit.default_timer()
try:
    del(df_train_iter)    
except NameError:
    pass

df_train_iter = pd.read_csv('../input/train.csv', chunksize=train_nrows_val//100,
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64},iterator=True) #use chunksize to iterate
df_before_jumping_up_points = pd.DataFrame()
for df in df_train_iter:
    if len(df.index.intersection(df_after_jumping_up_points.index-1)) > 0:
        try:
            df_before_jumping_up_points=df_before_jumping_up_points.append(df.loc[df.index.intersection(df_after_jumping_up_points.index-1),:])
        except KeyError:
            print('KeyError')
            pass
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))

In [ ]:
print(df_before_jumping_up_points.shape)
df_before_jumping_up_points

In [ ]:
start_time = timeit.default_timer()

try:
    del(df_train_tail)    
except NameError:
    pass
df_train_tail = pd.read_csv('../input/train.csv', skiprows = train_nrows_val-100000, iterator=False, names=column_names[0].split(','))
df_train_tail['acoustic_data'] = df_train_tail['acoustic_data'].astype(np.int16)
df_train_tail['time_to_failure'] = df_train_tail['time_to_failure'].astype(np.float64)
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))

In [ ]:
df_train_tail.tail()

In [ ]:
#steps in ttf:
#ttf steps width is 4097 for the last section:
#df_train_tail.tail(4097)['time_to_failure'].plot();
df_train_tail.tail(20000)['time_to_failure'].plot();

In [ ]:
start_time = timeit.default_timer()

try:
    del(df_train_head)    
except NameError:
    pass
df_train_head = pd.read_csv('../input/train.csv', skiprows = 0, nrows = 100000, iterator=False)

print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))

In [ ]:
#ttf steps width is 4096 for the first section:
#df_train_head.head(4096)['time_to_failure'].plot();
#df_train_head.head(20000)['time_to_failure'].plot();

df_train_head.head(20000)[8192:8192+4096]['time_to_failure'].plot();

In [ ]:
set(df_train_head.head(20000)[8192:8192+4095]['time_to_failure'].diff())

In [ ]:
n=0
set(df_train_head.head(2000000)['time_to_failure'].diff())

What these steps in ttf mean? <br>
Options: 
* 1. we are given not continuos in time sequencies (each ~4096 samples) of acustic data. 
* 2. the sequences of the acustic data are continuous in time but ttf represent changing steps/jumps/phases in the state of the material

The first option is more probable as it is unlikely that the changes in the phase states of the material will take place with such ideal periodicity. Then, time/frequency characteristics of the acustic_data should be considered taking into acount that the data is not continous in time. 

We assume that samples of the signal are taken each $10^{-3}$ time units (sec?/usec?) and each sample is a sequence of 4096 measurements taken each $10^{-9}$ time units. So, the length of the entire signal sample is $4.096\times10^{-6}$ time units

In [ ]:
index_ranges = [(ent[0],ent[1]) for ent in zip([0]+list(df_after_jumping_up_points.index)[:-1],list(df_before_jumping_up_points.index))]
index_ranges

In [ ]:
train_set_lengths =np.array([ent[1]-ent[0] for ent in zip([0]+list(df_before_jumping_up_points.index)[:-1],list(df_before_jumping_up_points.index))])
train_set_lengths

In [ ]:
train_set_lengths.mean()

In [ ]:
train_set_lengths.std()

In [ ]:
range_index = 3 #

In [ ]:
window_size = 15000
window_offset = -window_size
start_time = timeit.default_timer()
try:
    del(df_sample)    
except NameError:
    pass
df_sample = pd.read_csv('../input/train.csv', skiprows = index_ranges[range_index][0], nrows= index_ranges[range_index][1]-index_ranges[range_index][0],
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
df_sample.columns=['acoustic_data','time_to_failure']
print(df_sample.index)

fig, axs = plt.subplots(nrows=1, ncols=2, sharex=False)
fig.set_size_inches(32,4)
df_sample['acoustic_data'].plot(ax=axs[0]);
#plt.show()
df_sample['time_to_failure'].plot(ax=axs[1]);
#plt.show()
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))

We have 16 train sequences. We may use multiple ways to separate them into train and validation groups. This way we will be able to efficiently utilize the training data we have

In [ ]:
max_time_to_failure_points = pd.read_csv('../input/train.csv', skiprows = 0, nrows= 1, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})['time_to_failure'].append(df_after_jumping_up_points['time_to_failure'])
max_time_to_failure_points

In [ ]:
max_time_to_failure_points.values[:-1]

In [ ]:
decline_angle_tangents = np.array([ent[0]/ent[1] for ent in zip(max_time_to_failure_points.values[:-1], max_time_to_failure_points.index[1:])])
print(decline_angle_tangents.mean())
print(decline_angle_tangents.std())

In [ ]:
#keep minimal mem footprint 
try:
    del(df_sample)    
except NameError:
    pass

<a id='section4'></a>
## 4. Test Set EDA

In [ ]:
test_seg_files = listdir("../input/test")
test_seg_files[:5]

In [ ]:
#!ls -l ../input/test | wc -l
len(test_seg_files)

In [ ]:
os.path.join("../input/test",test_seg_files[0])

In [ ]:
!wc -l {os.path.join("../input/test",test_seg_files[0])}

In [ ]:
!head {os.path.join("../input/test",test_seg_files[0])}

In [ ]:
def plot_test_seg_by_index(idx):
    df_test_seg = pd.read_csv(os.path.join("../input/test",test_seg_files[idx]), dtype={'acoustic_data': np.int16})
    (df_test_seg['acoustic_data']-df_test_seg['acoustic_data'].mean()).plot();

In [ ]:
interact(plot_test_seg_by_index, idx=widgets.IntSlider(min=0,max=len(test_seg_files)-1,step=1,value=0));

In [ ]:
#make sure that all seg files have the same length: 150,000 samples:
seg_files_lengths = !for filename in ../input/test/*; do wc -l $filename; done
{ent.split(' ')[0] for ent in seg_files_lengths}

time length of a test segment defines meaninful precision of the predicted ttf

Question: assuming that each test segment contains samples without long jumps after each 4096 samples, what is the time length of the test segment? 

In [ ]:
(max(time_diff_within_sampling_frames_2_count_dict.keys())/(10.**max_precision)*150000)

Question: assuming that the test segments contain long jupmps after each 4096 samples, what is the time length of the test segment? 

In [ ]:
np.mean(list(time_diff_between_sampling_frames_2_count_dict.keys()))*(150000/4096)

<a id='section5'></a>
## 5. Models

Try the following:
* 0. collect features per sampling frame (4095, 4096, 8192 samples)
* 1. investigate train sequences in parallel with window of variable size:  15000 or smaller
    * 1.1 normalize to the same process development speed; (with_time_to_failure plots)
    * 1.2 present the acustic_data in a form of mean plus delta from mean, then delta from mean present by its mean and delta from its mean etc. 
* 2. remove constant bias in train and in test
* 3. simple MSE alignment 
* 4. simple maximaize dot product
* 5. fourier (fft)
    * 5.1 look at differences in fourier coefficients between the sampling frames
* 6. short-time fourier (time frequency domain)
* 7. wavelets (time frequency domain)
    * 7.1. cwt and dwt
    * 7.2. Haar (rectangular) and Morlet wavelets
* 8. voice spectrogrum; what is we are already given a spectorgrum (or other type of transform) but not the original signal? 
* 9. try to use notebooks used for the detection of gravitation waves
* 10. xgboost
* 11. try to extract a "coherent" signal and to remove the noise in the training data sequences 
* 12. set evaluation criterion
* 13. try to accelerate the process using pytorch (on gpu) for matrix and vector operations
* 14. try Fully Connected NN
* 14. try CNNs on 64 x 64 images based on 4096 samples frames
* 15. try RNNs/LSTMs
* 16. try exponential or other smoothing to reduce noise 
* 17. try considering 4096 samples as 64 x 64 image
* 18. try averaging over all 16 training sequences - (in time or frequencey space)

<a id='section5.1'></a>
### 5.1. Correlations

In [ ]:
start_time = timeit.default_timer()
try:
    del(df_sample)    
except NameError:
    pass

range_index=0 #first training sequence is the shortest one
df_sample = pd.read_csv('../input/train.csv', skiprows = index_ranges[range_index][0], nrows= index_ranges[range_index][1]-index_ranges[range_index][0],
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
df_sample.columns=['acoustic_data','time_to_failure']

fig, axs = plt.subplots(nrows=1, ncols=2, sharex=False)
fig.set_size_inches(32,4)
df_sample['acoustic_data'].plot(ax=axs[0]);
#plt.show()
print ('time_to_failure decline rate = {:.16f}'.format(df_sample['time_to_failure'][0]/df_sample['time_to_failure'].shape[0]))
df_sample['time_to_failure'].plot(ax=axs[1]);
#plt.show()
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))

In [ ]:
df_sample['time_to_failure'][0]/df_sample['time_to_failure'].shape[0]

In [ ]:
df_sample['time_to_failure'].shape[0]

we need to find points of maximum cross-correlation between the segment and the training sequence<br>
To save time, the correlation should be calculated efficiently - this means do not implement it by yourself in python. We need to find best implementation of the correlation function. 

In [ ]:
df_sample['acoustic_data'].head()

In [ ]:
df_sample['acoustic_data'].mean()

In [ ]:
train_values = (df_sample['acoustic_data']-df_sample['acoustic_data'].mean()).values
train_values

In [ ]:
df_test_seg = pd.read_csv(os.path.join("../input/test",test_seg_files[0]), dtype={'acoustic_data': np.int16})
df_test_seg.head()

In [ ]:
df_test_seg['acoustic_data'].mean()

In [ ]:
test_values = (df_test_seg['acoustic_data']-df_test_seg['acoustic_data'].mean()).values

In [ ]:
from scipy import signal

In [ ]:
train_values.shape

In [ ]:
test_values.shape

In [ ]:
signal_corr = signal.correlate(np.square(train_values), np.square(test_values),mode='valid', #full, valid, same
                               method='fft')

In [ ]:
signal_corr.shape

In [ ]:
pd.DataFrame(signal_corr).plot();

In [ ]:
def correlation_with_test_seg_idx(idx):
    df_test_seg = pd.read_csv(os.path.join("../input/test",test_seg_files[idx]), dtype={'acoustic_data': np.int16})
    test_values = (df_test_seg['acoustic_data']-df_test_seg['acoustic_data'].mean()).values
    signal_corr = signal.correlate(np.square(train_values), np.square(test_values),mode='same', method='fft')
    pd.DataFrame(signal_corr).plot();

In [ ]:
interact(correlation_with_test_seg_idx, idx=widgets.IntSlider(min=0,max=len(test_seg_files)-1,step=1,value=0));

In [ ]:
#start_time = timeit.default_timer()
#np_corr = np.correlate(df_sample['acoustic_data'].values, df_test_seg['acoustic_data'].values)
#print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))    
#this takes too long

<a id='section5.2'></a>
### 5.2 Spectrogram

In [ ]:
from scipy.signal import spectrogram

In [ ]:
M = 1024
N = 1024
freqs, times, Sx = signal.spectrogram(df_test_seg['acoustic_data'].values, fs=1, window='hanning',
                                      nperseg=N, noverlap=M - 100,
                                      detrend=False, scaling='spectrum')

f, ax = plt.subplots(figsize=(4.8, 2.4))
ax.pcolormesh(times, freqs / 1000, 10 * np.log10(Sx), cmap='viridis')
ax.set_ylabel('Frequency [kHz]')
ax.set_xlabel('Time [s]');

In [ ]:
f, t, Sxx = spectrogram(df_test_seg['acoustic_data'].values)
plt.pcolormesh(t, f, 10 * np.log10(Sxx))
plt.show()
plt.plot(Sxx)
#plt.ylabel('Frequency [Hz]')
#plt.xlabel('Time [sec]')
plt.show()

In [ ]:
from skimage import util

M = 1024

slices = util.view_as_windows(df_test_seg['acoustic_data'].values, window_shape=(M,), step=100)
print(f'data shape: {df_test_seg["acoustic_data"].values.shape}, Sliced data shape: {slices.shape}')

In [ ]:
win = np.hanning(M + 1)[:-1]
slices = slices * win

In [ ]:
slices = slices.T
print('Shape of `slices`:', slices.shape)

In [ ]:
spectrum = np.fft.fft(slices, axis=0)[:M // 2 + 1:-1]
spectrum = np.abs(spectrum)

In [ ]:
L=df_test_seg['acoustic_data'].values.shape[0]

In [ ]:
rate=10

In [ ]:
f, ax = plt.subplots(figsize=(4.8, 2.4))

S = np.abs(spectrum)
S = 20 * np.log10(S / np.max(S))

ax.imshow(S, origin='lower', cmap='viridis',
          extent=(0, L, 0, rate / 2 / 1000))
ax.axis('tight')
ax.set_ylabel('Frequency [kHz]')
ax.set_xlabel('Time [s]');

In [ ]:
#df_test_seg['acoustic_data'].values
# Number of sample points
#N = 600
# sample spacing
#T = 1.0 / 800.0
#x = np.linspace(0.0, N*T, N)
#y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)

#yf = fft(y)
yf = fftpack.fft(df_test_seg['acoustic_data'].values)
#df_test_seg['acoustic_data'].values

#xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

#plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.plot(np.abs(yf))
plt.grid()
plt.show()

<a id='section5.3'></a>
## 5.3 Fourier Transform per Sampling Sequence of 4096 measurements

https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.fft.html <br>
https://docs.scipy.org/doc/scipy/reference/tutorial/fftpack.html

In [ ]:
display_df_with_preset_precision(df_after_long_jumps_down_points.head(),max_precision)

In [ ]:
range_index=12 #first training sequence is the shortest one
#sequence_length = 4096
start_time = timeit.default_timer()
try:
    del(df_sample)    
except NameError:
    pass
df_sample = pd.read_csv('../input/train.csv', skiprows = index_ranges[range_index][0], nrows= index_ranges[range_index][1]-index_ranges[range_index][0],
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
df_sample.columns=['acoustic_data','time_to_failure']
#print(df_sample.shape[0]/sequence_length)
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))

In [ ]:
#display_df_with_preset_precision(df_sample.head(),max_precision)
df_sample.index

In [ ]:
index_ranges[range_index]

In [ ]:
ixs_of_indexes = np.where(np.all([df_after_long_jumps_down_points.index <= index_ranges[range_index][1],df_after_long_jumps_down_points.index >= index_ranges[range_index][0]-1],axis=0))[0]
ixs_of_indexes

In [ ]:
ixs_of_indexes1 = np.where(np.logical_and(df_after_long_jumps_down_points.index <= index_ranges[range_index][1],
                        df_after_long_jumps_down_points.index >= index_ranges[range_index][0]))[0]
ixs_of_indexes1

In [ ]:
np.array_equal(ixs_of_indexes1,ixs_of_indexes)

In [ ]:
indexes = df_after_long_jumps_down_points.index[ixs_of_indexes].union(index_ranges[range_index])

In [ ]:
np.unique(np.diff(indexes))

In [ ]:
frame_indexes = list(zip(indexes[:-1],indexes[1:]))

In [ ]:
len(list(frame_indexes))

TBD: work with resolution of a sampling frame <br>
for each sampling frame:
* TBD: calculate the average ttf - the label
* TBD: check what is the time difference between near samples - check if we can find any pattern
* TBD: take an average amplitude of the signal 
* TBD: get fft 
* TBD: get rfft 
* TBD: get differences between fft coefficients between consequent frames

In [ ]:
#MEAN (Bias) is removed 
low_path_filter_n_freqs = 1024
avg_len=1
frame_sequence_offset = -frame_indexes[0][0]
def show_frame_and_fft(sequence_idx):
    fig, axs = plt.subplots(nrows=1, ncols=3, sharex=False)

    #print(df_sample[frame_sequence_offset+sequence_length*sequence_idx:frame_sequence_offset+sequence_length*sequence_idx+sequence_length].shape)
    the_df = df_sample[frame_sequence_offset+frame_indexes[sequence_idx][0]:frame_sequence_offset+frame_indexes[sequence_idx][1]]
    print(the_df.shape)
    #df_sample[frame_sequence_offset+sequence_length*sequence_idx:frame_sequence_offset+sequence_length*sequence_idx+sequence_length]['time_to_failure'].plot(ax=axs[0]);
    #the_df['time_to_failure'].plot(ax=axs[0]);
    diff_ttf = the_df['time_to_failure'].diff()[1:]
    
    #acustic_data_series = df_sample[frame_sequence_offset+sequence_length*sequence_idx:frame_sequence_offset+sequence_length*sequence_idx+sequence_length]['acoustic_data']
    acustic_data_series = the_df['acoustic_data']
    
    print('ttf = {:.9f}'.format(the_df['time_to_failure'].mean()))
    print('diff_ttf.mean = {:.9f}, diff_ttf.std = {:.9f}'.format(diff_ttf.mean(), diff_ttf.std()))
    print('acustic.mean = {:.9f}, acustic.std = {:.9f}'.format(acustic_data_series.mean(), acustic_data_series.std()))
    
    (acustic_data_series - acustic_data_series.mean()).plot(ax=axs[0]);
    #df_sample[frame_sequence_offset+sequence_length*sequence_idx:frame_sequence_offset+sequence_length*sequence_idx+sequence_length]['acoustic_data'].plot(ax=axs[1]);

    fig.set_size_inches(32,4)
    
    acustic_data_rft = fftpack.rfft(acustic_data_series-acustic_data_series.mean())
    acustic_data_ft = fftpack.fft(acustic_data_series-acustic_data_series.mean())
    
    rfreqs = fftpack.rfftfreq(acustic_data_rft.size,diff_ttf.mean())
    freqs = fftpack.fftfreq(acustic_data_ft.size, diff_ttf.mean())
    
    
    #freqs = freqs/(diff_ttf.mean()*the_df.shape[0])
    rfreqs = -rfreqs
    
    the_dict = {}
    for idx in range(len(acustic_data_rft)):
        if rfreqs[idx] in the_dict:
            the_dict[rfreqs[idx]] = (the_dict[rfreqs[idx]]+np.abs(acustic_data_rft[idx]))/2.
        else: 
            the_dict[rfreqs[idx]] = np.abs(acustic_data_rft[idx])
    unique_rfreqs = np.unique(rfreqs)
    print("arrays are equal is {}".format(np.array_equal(sorted(unique_rfreqs),unique_rfreqs)))
    
    #print(len(rfreqs[:]))
    #print(type(rfreqs[:]))
    #print(len(np.unique(rfreqs[:])))
    #print(freqs[len(freqs)//2:])
    #pd.DataFrame.from_dict({'acustic_data_ft_amp': np.abs(acustic_data_ft)[len(freqs)//2:], 'freqs':freqs[len(freqs)//2:]}).set_index('freqs').plot(ax=axs[2])
    #pd.DataFrame.from_dict({'acustic_data_rft_amp': [the_dict[ent] for ent in unique_rfreqs], 'rfreqs':unique_rfreqs}).set_index('rfreqs').plot(ax=axs[1])
    pd.DataFrame.from_dict({'acustic_data_rft_amp': [the_dict[ent] for ent in unique_rfreqs][:low_path_filter_n_freqs], 
                            'rfreqs':unique_rfreqs[:low_path_filter_n_freqs]}).set_index('rfreqs').plot(ax=axs[1])
    #pd.DataFrame.from_dict({'acustic_data_rft_amp': np.abs(acustic_data_rft), 'rfreqs':rfreqs}).set_index('rfreqs').plot(ax=axs[1])
    pd.DataFrame.from_dict({'acustic_data_ft_amp': (np.abs(acustic_data_ft)[len(freqs)//2:]), 
                            'freqs':freqs[len(freqs)//2:]}).set_index('freqs').plot(ax=axs[2])

    
    
    #pd.DataFrame.from_dict({'acustic_data_ft_angle': np.angle(acustic_data_ft), 'freqs':freqs}).set_index('freqs').plot(ax=axs[3])
    #pd.DataFrame.from_dict({'acustic_data_rft_angle': np.angle(acustic_data_rft), 'rfreqs':rfreqs}).set_index('rfreqs').plot(ax=axs[3])
    #pd.DataFrame.from_dict({'acustic_data_ft_angle': np.angle(acustic_data_ft)[1:]+np.angle(acustic_data_ft)[:len(acustic_data_ft)-1], 'freqs':freqs[1:]}).set_index('freqs').plot(ax=axs[3])
    #pd.DataFrame.from_dict({'acustic_data_ft_angle': np.angle(acustic_data_ft), 'freqs':freqs}).set_index('freqs').rolling(avg_len).mean().plot(ax=axs[3])

    #pd.DataFrame.from_dict({'acustic_data_ft_real': np.real(acustic_data_ft), 'freqs':freqs}).set_index('freqs').plot(ax=axs[4])

    #pd.DataFrame.from_dict({'acustic_data_ft_img': np.imag(acustic_data_ft), 'freqs':freqs}).set_index('freqs').plot(ax=axs[5])

    #axs[2].stem(freqs, np.abs(acustic_data_ft))
    #len(acustic_data_rft)
    #print ('time_to_failure decline rate = {:.16f}'.format(df_sample['time_to_failure'][0]/df_sample['time_to_failure'].shape[0]))

In [ ]:
print('range_index = {}'.format(range_index))
interact(show_frame_and_fft, sequence_idx=widgets.IntSlider(min=0,max=len(list(frame_indexes))-1,step=1,value=0));

In [ ]:
def calc_fft_amp_per_sequence_index(sequence_idx):
    the_df = df_sample[frame_sequence_offset+frame_indexes[sequence_idx][0]:frame_sequence_offset+frame_indexes[sequence_idx][1]]

    diff_ttf = the_df['time_to_failure'].diff()[1:]   
    acustic_data_series = the_df['acoustic_data']
    
    print('ttf = {:.9f}'.format(the_df['time_to_failure'].mean()))
    print('diff_ttf.mean = {:.9f}, diff_ttf.std = {:.9f}'.format(diff_ttf.mean(), diff_ttf.std()))
    print('acustic.mean = {:.9f}, acustic.std = {:.9f}'.format(acustic_data_series.mean(), acustic_data_series.std()))    
    
    acustic_data_rft = fftpack.rfft(acustic_data_series-acustic_data_series.mean())
       
    rfreqs = -fftpack.rfftfreq(len(acustic_data_rft),diff_ttf.mean())   
    
    the_dict = {}
    for idx in range(len(acustic_data_rft)):
        if rfreqs[idx] in the_dict:
            the_dict[rfreqs[idx]] = (the_dict[rfreqs[idx]]+np.abs(acustic_data_rft[idx]))/2.
        else: 
            the_dict[rfreqs[idx]] = np.abs(acustic_data_rft[idx])
    
    unique_rfreqs = np.unique(rfreqs)
    
    return pd.DataFrame.from_dict({'acustic_data_rft_amp': [the_dict[ent] for ent in unique_rfreqs][:low_path_filter_n_freqs], 
                                   'rfreqs':unique_rfreqs[:low_path_filter_n_freqs]}).set_index('rfreqs')

In [ ]:
(calc_fft_amp_per_sequence_index(0)['acustic_data_rft_amp']).values[:low_path_filter_n_freqs]

In [ ]:
def features_row_per_sequence_index(sequence_idx):
    the_df = df_sample[frame_sequence_offset+frame_indexes[sequence_idx][0]:frame_sequence_offset+frame_indexes[sequence_idx][1]]

    diff_ttf = the_df['time_to_failure'].diff()[1:]   
    acustic_data_series = the_df['acoustic_data']
    
    #print('ttf = {:.9f}'.format(the_df['time_to_failure'].mean()))
    #print('diff_ttf.mean = {:.9f}, diff_ttf.std = {:.9f}'.format(diff_ttf.mean(), diff_ttf.std()))
    #print('acustic.mean = {:.9f}, acustic.std = {:.9f}'.format(acustic_data_series.mean(), acustic_data_series.std()))    
    
    acustic_data_rft = fftpack.rfft(acustic_data_series-acustic_data_series.mean())
       
    rfreqs = -fftpack.rfftfreq(len(acustic_data_rft),diff_ttf.mean())   
    
    the_dict = {}
    for idx in range(len(acustic_data_rft)):
        if rfreqs[idx] in the_dict:
            the_dict[rfreqs[idx]] = (the_dict[rfreqs[idx]]+np.abs(acustic_data_rft[idx]))/2.
        else: 
            the_dict[rfreqs[idx]] = np.abs(acustic_data_rft[idx])
    
    unique_rfreqs = np.unique(rfreqs)
    
    return [acustic_data_series.mean()]+[the_dict[ent] for ent in unique_rfreqs][:low_path_filter_n_freqs][1:]+[the_df['time_to_failure'].mean()]

In [ ]:
len(features_row_per_sequence_index(0))

<a id='section5.4'></a>
## 5.4 Wavelets Transform per Sampling Sequence of 4096 measurements

In [ ]:
range_index=12 #first training sequence is the shortest one
sequence_length = 4096
start_time = timeit.default_timer()
try:
    del(df_sample)    
except NameError:
    pass
df_sample = pd.read_csv('../input/train.csv', skiprows = index_ranges[range_index][0], nrows= index_ranges[range_index][1]-index_ranges[range_index][0],
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
df_sample.columns=['acoustic_data','time_to_failure']
print(df_sample.shape[0]/sequence_length)
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))

https://github.com/fbcotter/pytorch_wavelets

[Drop an Octave: Reducing Spatial Redundancy in Convolutional Neural Networks with Octave Convolution](https://arxiv.org/abs/1904.05049)

In [ ]:
#from scipy.signal import cwt
from scipy import signal



<a id='section5.4.1'></a>
### 5.4.1. Continous Wavelet Transform

In [ ]:
#MEAN (Bias) is removed 
#sequence_idx = 2 #10845
avg_len=2
sequence_offset = 0
def show_frame_and_cwt(sequence_idx):
    fig, axs = plt.subplots(nrows=1, ncols=3, sharex=False)

    print(df_sample[sequence_offset+sequence_length*sequence_idx:sequence_offset+sequence_length*sequence_idx+sequence_length].shape)
    df_sample[sequence_offset+sequence_length*sequence_idx:sequence_offset+sequence_length*sequence_idx+sequence_length]['time_to_failure'].plot(ax=axs[0]);
    
    acustic_data_series = df_sample[sequence_offset+sequence_length*sequence_idx:sequence_offset+sequence_length*sequence_idx+sequence_length]['acoustic_data']
    acustic_data_series_zero_mean = (acustic_data_series - acustic_data_series.mean())
    acustic_data_series_zero_mean.plot(ax=axs[1])
    #df_sample[sequence_offset+sequence_length*sequence_idx:sequence_offset+sequence_length*sequence_idx+sequence_length]['acoustic_data'].plot(ax=axs[1]);

    fig.set_size_inches(32,4)
    
    acustic_data_cwt = signal.cwt(acustic_data_series_zero_mean, signal.morlet, #signal.morlet, signal.ricker
                                  np.arange(1,31))
    axs[2].imshow(acustic_data_cwt, #extent=[-1,1,31,1], 
                  cmap='PRGn', aspect='auto',  #vmax=abs(acustic_data_series_zero_mean).max(), vmin=-abs(acustic_data_series_zero_mean).max()
                 )
    #axs[2].stem(freqs, np.abs(acustic_data_ft))
    print(len(acustic_data_cwt))
    #print ('time_to_failure decline rate = {:.16f}'.format(df_sample['time_to_failure'][0]/df_sample['time_to_failure'].shape[0]))
    print(acustic_data_cwt.shape)

In [ ]:
interact(show_frame_and_cwt, sequence_idx=widgets.IntSlider(min=0,max=df_sample.shape[0]/sequence_length,step=1,value=0));

<a id='section5.4.2'></a>
### 5.4.2. Discrete Wavelet Transform

Denoise and compress the signal

multilevel wavelet decomposition

In [ ]:
from pywt import wavedec

In [ ]:
#MEAN (Bias) is removed 
#sequence_idx = 2 #10845
avg_len=2
sequence_offset = 0
def show_frame_and_dwt(sequence_idx):
    fig, axs = plt.subplots(nrows=1, ncols=2, sharex=False)

    print(df_sample[sequence_offset+sequence_length*sequence_idx:sequence_offset+sequence_length*sequence_idx+sequence_length].shape)
    df_sample[sequence_offset+sequence_length*sequence_idx:sequence_offset+sequence_length*sequence_idx+sequence_length]['time_to_failure'].plot(ax=axs[0]);
    
    acustic_data_series = df_sample[sequence_offset+sequence_length*sequence_idx:sequence_offset+sequence_length*sequence_idx+sequence_length]['acoustic_data']
    acustic_data_series_zero_mean = (acustic_data_series - acustic_data_series.mean())
    acustic_data_series_zero_mean.plot(ax=axs[1])
    #df_sample[sequence_offset+sequence_length*sequence_idx:sequence_offset+sequence_length*sequence_idx+sequence_length]['acoustic_data'].plot(ax=axs[1]);

    fig.set_size_inches(32,4)
    
    acustic_data_dwt_coeffs = wavedec(acustic_data_series_zero_mean,'db1', level=1)
    #acustic_data_cwt = signal.cwt(acustic_data_series_zero_mean, signal.morlet, #signal.morlet, signal.ricker
    #                              np.arange(1,31))
    #axs[2].imshow(acustic_data_cwt, #extent=[-1,1,31,1], 
    #              cmap='PRGn', aspect='auto',  #vmax=abs(acustic_data_series_zero_mean).max(), vmin=-abs(acustic_data_series_zero_mean).max()
    #             )
    #axs[2].stem(freqs, np.abs(acustic_data_ft))
    print(len(acustic_data_dwt_coeffs))
    print((acustic_data_dwt_coeffs[0]).shape)
    #print ('time_to_failure decline rate = {:.16f}'.format(df_sample['time_to_failure'][0]/df_sample['time_to_failure'].shape[0]))
    print(acustic_data_dwt_coeffs)

In [ ]:
interact(show_frame_and_dwt, sequence_idx=widgets.IntSlider(min=0,max=df_sample.shape[0]/sequence_length,step=1,value=0));

<a id='section6'></a>
## Evaluation

given the test segment length 150000 samples, what is the required time precision?